# LAB GenAI - LLMs - OpenAI Assistant

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

I created a market research assistant that provides basic information about the types of market research studies, kinds of questions each answers, etc. using the OpenAI API platform.

**Uploaded File**
The file provided to the assistant is a pdf with basic information about market research. It can be accessed at https://www.denefield.org.uk/_site/data/files/users/91/files/CFFA56541EE81FF71B9A0CA892F27013.pdf

**System Instructions**
### ROLE:
You are a highly experienced Market Research Consultant specializing in types of market research.

### OBJECTIVE:
- Help analyze and interpret market trends.
- Assist in recommending type(s) of research to conduct based on the user's problem.

### INSTRUCTIONS:
- When asked about general questions about market research, use the provided document as the main source.
- Answer in a professional and concise manner.
- If the information is not available, ask clarifying questions before making assumptions.

## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [43]:
# Asks the assistant a question via the API.
# Assistant specifics can be found under Exercise 1.

# Loads the necessary libraries
import os
import openai
import time
from IPython.display import display, Markdown

# Before executing the following lines, make sure to open Terminal and enter export OPENAI_API_KEY="your api key here"
# Loads the API Key from environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Checks if the key exists before proceeding
if not OPENAI_API_KEY:
    raise ValueError("🚨 OPENAI_API_KEY is not set! Please configure your environment variables.")

# Initializes the OpenAI Client
client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [21]:
# Defines the Assistant ID
# The ASSISTANT_ID can be found within the dashboard in the OpenAI API platform.
ASSISTANT_ID = "asst_W0whYK3RM4lfn7nMpbf9nvx6"  

# Creates a new conversation thread
thread = client.beta.threads.create()

# Sends a message to the assistant
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What are some examples of quantitative market research?"
)

# Runs the assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=ASSISTANT_ID
)

# Waits for the response
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    time.sleep(1)  # Wait a second before checking again

# Retrieves response messages
messages = client.beta.threads.messages.list(thread_id=thread.id)

# Extracts and formats the assistant's response
for msg in messages.data:
    if msg.role == "assistant":
        response_text = msg.content[0].text.value  # Extracts text content

        # Apply Markdown formatting
        formatted_response = f"### 🤖 Assistant Response:\n\n{response_text}"

        # Display the response in a readable format
        display(Markdown(formatted_response))  # Renders Markdown

### 🤖 Assistant Response:

Quantitative market research generally involves collecting and analyzing numerical data to understand market trends, consumer behavior, and preferences quantifiably. Here are some common examples of quantitative market research methods:

1. **Surveys:** Structured questionnaires that include closed-ended questions. They can be delivered online, by phone, via email, or in person, and are useful for gathering large volumes of data from a significant sample size.

2. **Polls:** Shorter than surveys, polls quickly gather opinions by asking one or more multiple-choice questions to a large audience. They are useful for gauging public opinion on specific issues.

3. **Experiments/Field Trials:** Controlled tests where variables are manipulated to see effects on subjects (e.g., consumer response to new packaging or formulations). These can give clear, actionable results.

4. **Observational Studies:** Data is collected just by observing subjects in natural or controlled environments, often automated via technology (e.g., tracking customer movements in stores through cameras).

5. **Statistical Analysis:** Application of statistical techniques to analyze collected data, which can reveal patterns, correlations, and insights in large datasets.

6. **Longitudinal Studies:** Studies conducted over a long period to observe market trends and behaviors over time. They help identify long-term changes and effects.

Using these methods, companies can quantify preferences, measure effects of marketing campaigns, forecast market trends, and make data-driven decisions.

## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [37]:
# Creates an assistant that will call a weather API, given the user's answer and return the proper answer.
# Imports necessary libraries
import openai
import requests
import os
import time
import re
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Loads environment variables from .env file
load_dotenv()

# Sets the API keys
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY NOT Found! Please check your .env file.")

# Initializes the OpenAI Client
client = openai.OpenAI(api_key=api_key)

# Defines the Assistant ID 
ASSISTANT_ID = "asst_W0whYK3RM4lfn7nMpbf9nvx6"

# Defines a function to call Open-Meteo API for weather forecast
def get_weather_forecast(latitude, longitude):
    """
    Calls Open-Meteo API to fetch current weather data.
    """
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "current_weather": "true"
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Unable to fetch weather data."}

# Defines a function to get coordinates from OpenAI
def get_coordinates_from_openai(location):
    """
    Queries OpenAI Assistant to return the latitude and longitude for a given city.
    """
    query = f"Provide ONLY the latitude and longitude for {location} in the format: 'latitude, longitude'. No extra text."

    # Creates a conversation thread
    thread = client.beta.threads.create()
    
    # Sends user query
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=query
    )

    # Runs the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=ASSISTANT_ID
    )

    # Waits for OpenAI response
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status == "completed":
            break
        time.sleep(1)  # Wait a second before checking again

    # Retrieves response messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)

    # Extracts assistant response
    response_text = ""
    for msg in messages.data:
        if msg.role == "assistant":
            response_text = msg.content[0].text.value
            break

    # Extracts the latitude & longitude from response
    coords = re.findall(r"[-+]?\d*\.\d+|\d+", response_text)

    if len(coords) >= 2:
        return float(coords[0]), float(coords[1])  # Convert to float
    else:
        raise ValueError(f"Could not extract latitude/longitude from response: {response_text}")

# User Input for Location
user_location = input("Enter a city or coordinates (latitude, longitude): ").strip()

# Checks if user entered raw coordinates (e.g., "18.4655, -66.1057")
try:
    if "," in user_location:
        lat, lon = map(float, user_location.split(","))
        print(f"Using provided coordinates: Latitude={lat}, Longitude={lon}")
    else:
        raise ValueError  # Force the exception to fetch coordinates via OpenAI
except ValueError:
    print("Detected city name, fetching latitude/longitude from OpenAI...")
    lat, lon = get_coordinates_from_openai(user_location)
    print(f"Extracted Coordinates: Latitude={lat}, Longitude={lon}")

# Calls the weather API
weather_data = get_weather_forecast(lat, lon)

# Displays the result
if "error" in weather_data:
    print("Error fetching weather data.")
else:
    current_temp = weather_data["current_weather"]["temperature"]
    formatted_response = f"### 🌤️ Weather Forecast for {user_location}:\n- **Current Temperature**: {current_temp}°C"

    # Displays the output in markdown format
    display(Markdown(formatted_response))

Enter a city or coordinates (latitude, longitude):  San Juan, Puerto Rico


📍 Detected city name, fetching latitude/longitude from OpenAI...
✅ Extracted Coordinates: Latitude=18.4655, Longitude=-66.1057


### 🌤️ Weather Forecast for San Juan, Puerto Rico:
- **Current Temperature**: 24.7°C

### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality